In [2]:
# Imports
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, GeoAccessor, FeatureSet, GeoSeriesAccessor

### ACS Data

In [3]:
# Extract Maryland ACS 5-year data profiles containing broad social, economic, housing, and demographic information. 
ACS_df=pd.read_json('https://api.census.gov/data/2018/acs/acs5/profile?get=DP05_0024E,DP05_0024PE,DP02_0112E,DP02_0112PE,DP02_0152E,DP02_0152PE,DP02_0061E,DP02_0061PE,DP02_0071E,DP02_0071PE,DP03_0128PE&for=tract:*&in=state:24')
ACS_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,DP05_0024E,DP05_0024PE,DP02_0112E,DP02_0112PE,DP02_0152E,DP02_0152PE,DP02_0061E,DP02_0061PE,DP02_0071E,DP02_0071PE,DP03_0128PE,state,county,tract
1,902,15.8,987,17.4,2858,76.5,307,6.5,722,12.7,19.5,24,510,110200
2,317,8.2,475,12.8,1900,88.7,283,8.1,277,7.1,9.3,24,510,020300
3,593,14.9,194,5.0,761,62.4,857,35.8,717,18.0,21.4,24,510,150400
4,229,8.0,239,8.9,1214,87.2,231,9.7,221,7.7,5.9,24,510,010200


In [4]:
# Clean and query data to desired form with pandas.
ACS_df.columns = ACS_df.iloc[0]
ACS_df.drop(ACS_df.index[0])

,DP05_0024E,DP05_0024PE,DP02_0112E,DP02_0112PE,DP02_0152E,DP02_0152PE,DP02_0061E,DP02_0061PE,DP02_0071E,DP02_0071PE,DP03_0128PE,state,county,tract
1,902,15.8,987,17.4,2858,76.5,307,6.5,722,12.7,19.5,24,510,110200
2,317,8.2,475,12.8,1900,88.7,283,8.1,277,7.1,9.3,24,510,020300
3,593,14.9,194,5.0,761,62.4,857,35.8,717,18.0,21.4,24,510,150400
4,229,8.0,239,8.9,1214,87.2,231,9.7,221,7.7,5.9,24,510,010200
5,188,13.7,285,21.8,547,71.9,210,18.7,223,16.2,14.4,24,510,060400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402,190,3.2,1789,30.1,1494,91.2,60,3.2,303,5.1,35.6,24,510,120202
1403,1016,26.1,508,14.7,1408,76.4,652,23.9,639,16.4,12.6,24,510,272004
1404,614,18.1,938,30.0,840,67.1,334,15.3,427,12.6,19.4,24,510,272005
1405,856,24.2,381,12.0,899,55.4,733,30.7,771,21.9,15.5,24,510,272006


In [5]:
# filter down to eastern shore counties
easternMD_ACS_df = ACS_df[ACS_df['county'].isin(['019', '015', '029', '011', '047', '045', '039', '035', '041'])]
easternMD_ACS_df

,DP05_0024E,DP05_0024PE,DP02_0112E,DP02_0112PE,DP02_0152E,DP02_0152PE,DP02_0061E,DP02_0061PE,DP02_0071E,DP02_0071PE,DP03_0128PE,state,county,tract
6,150,3.8,150,3.8,25,100.0,1727,50.5,33,57.9,0.0,24,039,980400
7,599,34.5,38,2.2,591,78.3,277,19.4,265,15.3,4.4,24,041,960600
8,1505,43.6,97,2.9,1475,85.5,452,15.4,505,14.6,4.8,24,041,960700
9,1074,20.4,479,9.7,1972,77.7,1009,27.0,755,15.2,13.3,24,041,960400
10,711,18.4,733,20.6,1422,80.9,908,34.1,728,19.2,13.0,24,041,960300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396,814,24.0,77,2.4,882,67.8,996,40.4,684,20.4,10.4,24,047,951400
1397,888,19.8,155,3.7,1225,67.1,1262,41.4,871,19.7,24.7,24,047,951500
1398,961,26.5,220,6.2,1279,81.6,1020,34.8,523,14.5,9.2,24,047,951700
1399,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,-666666666.0,24,047,980000


In [5]:
# Replace column names and sort by 2013-2018 Total Percent of population 65 years and older.
easternMD_ACS_df = (easternMD_ACS_df
          .rename(index=str, 
                  columns={'DP05_0024E': 'TotEst_65Years', 
                           'DP05_0024PE': 'TotPerc_65Years',
                           'DP02_0112E': 'TotEst_LangNotEnglish',
                           'DP02_0112PE': 'TotPercLangNotEnglish',
                           'DP02_0152E': 'TotEst_HHwithBroadband',
                           'DP02_0152PE': 'TotPerc_HHwithBroadband',
                           'DP02_0061E': 'TotEst_25YearsWithHSEd',
                           'DP02_0061PE': 'TotPerc_25YearsWithHSEd',
                           'DP02_0071E': 'TotEst_withDisability',
                           'DP02_0071PE': 'TotPerc_withDisability',
                           'DP03_0128PE': 'TotPerc_Poverty'})
          .sort_values(by='TotEst_HHwithBroadband', 
                       ascending=True)
                    .reset_index(drop=True))
easternMD_ACS_df[:20]

,TotEst_65Years,TotPerc_65Years,TotEst_LangNotEnglish,TotPercLangNotEnglish,TotEst_HHwithBroadband,TotPerc_HHwithBroadband,TotEst_25YearsWithHSEd,TotPerc_25YearsWithHSEd,TotEst_withDisability,TotPerc_withDisability,TotPerc_Poverty,state,county,tract
0,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,-666666666.0,24,047,990000
1,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,-666666666.0,24,041,990000
2,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,-666666666.0,24,039,990100
3,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,-666666666.0,24,035,990000
4,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,-666666666.0,24,035,990100
5,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,-666666666.0,24,035,990200
6,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,-666666666.0,24,029,990000
7,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,-666666666.0,24,047,980000
8,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,0,-666666666.0,-666666666.0,24,019,990000
9,457,14.1,162,5.3,1021,80.6,869,39.9,557,18.4,20.3,24,011,955302


In [6]:
# Drop zero values from analysis
easternMD_ACS_df = easternMD_ACS_df.drop([0,1,2,3,4,5,6,7,8], axis=0)

In [7]:
easternMD_ACS_df.head(20).sort_values(by='TotPerc_65Years', 
                       ascending=False).reset_index(drop=True)

,TotEst_65Years,TotPerc_65Years,TotEst_LangNotEnglish,TotPercLangNotEnglish,TotEst_HHwithBroadband,TotPerc_HHwithBroadband,TotEst_25YearsWithHSEd,TotPerc_25YearsWithHSEd,TotEst_withDisability,TotPerc_withDisability,TotPerc_Poverty,state,county,tract
0,1143,37.6,196,6.8,1214,88.7,468,19.3,406,13.4,6.6,24,047,950600
1,1282,32.4,171,4.5,1249,73.8,1102,34.1,807,21.3,9.7,24,019,970702
2,1037,24.6,131,3.2,1162,68.5,1146,38.1,671,16.2,10.5,24,039,930500
3,967,23.1,122,3.1,1254,72.9,1330,44.2,563,13.4,6.3,24,019,970300
4,809,22.4,428,12.3,1165,77.7,786,29.3,406,11.3,10.3,24,029,950100
5,792,20.5,177,4.8,1097,69.6,1257,42.7,625,16.7,15.2,24,039,930102
6,717,19.8,292,9.0,1173,80.1,698,28.7,363,10.1,15.9,24,045,010102
7,888,19.8,155,3.7,1225,67.1,1262,41.4,871,19.7,24.7,24,047,951500
8,583,19.6,105,3.7,1038,92.6,563,26.4,333,11.2,2.3,24,035,810902
9,626,17.1,174,5.0,1161,83.0,954,36.4,562,15.6,17.2,24,011,955201


In [8]:
easternMD_ACS_df.shape

(104, 14)

### 2020 Census Response Data

In [9]:
# Extract the Census response rate data from api (https://api.census.gov/data/2020/dec/responserate) into pandas df.
df=pd.read_json('https://api.census.gov/data/2020/dec/responserate?get=GEO_ID,NAME,CRRALL,DRRALL,CRRINT,DRRINT,RESP_DATE&for=tract:*&in=state:24')
df.head()

,0,1,2,3,4,5,6,7,8,9
0,GEO_ID,NAME,CRRALL,DRRALL,CRRINT,DRRINT,RESP_DATE,state,county,tract
1,1400000US24033806400,"Census Tract 8064, Prince George's County, Mar...",75.9,0.3,72,0.2,2020-08-18,24,033,806400
2,1400000US24033805907,"Census Tract 8059.07, Prince George's County, ...",48.1,0.3,40.7,0.2,2020-08-18,24,033,805907
3,1400000US24033805909,"Census Tract 8059.09, Prince George's County, ...",55.6,0.1,51.9,0,2020-08-18,24,033,805909
4,1400000US24033806100,"Census Tract 8061, Prince George's County, Mar...",68.6,0.4,62.4,0.1,2020-08-18,24,033,806100


In [10]:
# Clean and query data to desired form with pandas.
df.columns = df.iloc[0]
df.drop(df.index[0])

,GEO_ID,NAME,CRRALL,DRRALL,CRRINT,DRRINT,RESP_DATE,state,county,tract
1,1400000US24033806400,"Census Tract 8064, Prince George's County, Mar...",75.9,0.3,72,0.2,2020-08-18,24,033,806400
2,1400000US24033805907,"Census Tract 8059.07, Prince George's County, ...",48.1,0.3,40.7,0.2,2020-08-18,24,033,805907
3,1400000US24033805909,"Census Tract 8059.09, Prince George's County, ...",55.6,0.1,51.9,0,2020-08-18,24,033,805909
4,1400000US24033806100,"Census Tract 8061, Prince George's County, Mar...",68.6,0.4,62.4,0.1,2020-08-18,24,033,806100
5,1400000US24033806601,"Census Tract 8066.01, Prince George's County, ...",57,0,37.1,0,2020-08-18,24,033,806601
...,...,...,...,...,...,...,...,...,...,...
1452,1400000US24510260700,"Census Tract 2607, Baltimore city, Maryland",55.6,0.2,50.7,0.2,2020-08-18,24,510,260700
1453,1400000US24510261000,"Census Tract 2610, Baltimore city, Maryland",49.1,0,41.1,0,2020-08-18,24,510,261000
1454,1400000US24510270101,"Census Tract 2701.01, Baltimore city, Maryland",69,0,62.1,0,2020-08-18,24,510,270101
1455,1400000US24510271001,"Census Tract 2710.01, Baltimore city, Maryland",48.7,0.1,24.1,0.1,2020-08-18,24,510,271001


In [11]:
# filter down to eastern shore counties
easternMD_df = df[df['county'].isin(['019', '015', '029', '011', '047', '045', '039', '035', '041'])]
easternMD_df

,GEO_ID,NAME,CRRALL,DRRALL,CRRINT,DRRINT,RESP_DATE,state,county,tract
19,1400000US24035810100,"Census Tract 8101, Queen Anne's County, Maryland",51.9,0.4,25,0.3,2020-08-18,24,035,810100
20,1400000US24035810200,"Census Tract 8102, Queen Anne's County, Maryland",57.1,0.5,22.1,0,2020-08-18,24,035,810200
21,1400000US24035810400,"Census Tract 8104, Queen Anne's County, Maryland",69,0.3,60.5,0.2,2020-08-18,24,035,810400
22,1400000US24035810500,"Census Tract 8105, Queen Anne's County, Maryland",68,0.2,57.4,0.1,2020-08-18,24,035,810500
23,1400000US24035810700,"Census Tract 8107, Queen Anne's County, Maryland",62.9,0.2,55.2,0,2020-08-18,24,035,810700
...,...,...,...,...,...,...,...,...,...,...
1401,1400000US24045010503,"Census Tract 105.03, Wicomico County, Maryland",39.9,0.1,36.5,0.1,2020-08-18,24,045,010503
1402,1400000US24045010603,"Census Tract 106.03, Wicomico County, Maryland",63.7,0.2,55.4,0.1,2020-08-18,24,045,010603
1403,1400000US24045010606,"Census Tract 106.06, Wicomico County, Maryland",71.3,0.2,59.7,0.1,2020-08-18,24,045,010606
1404,1400000US24045010704,"Census Tract 107.04, Wicomico County, Maryland",73.7,0.3,60.9,0.1,2020-08-18,24,045,010704


In [12]:
# Replace column names.
easternMD_df = (easternMD_df
                .rename(index=str, 
                        columns={'CRRALL': 'CumulResponseRateOverall', 
                                 'DRRALL': 'DailyResponseRateOverall',
                                 'CRRINT': 'CumulResponseRateInternet',
                                 'DRRINT': 'DailyResponseRateInternet'})
                .sort_values(by='CumulResponseRateInternet', 
                             ascending=True))
easternMD_df[:20]

,GEO_ID,NAME,CumulResponseRateOverall,DailyResponseRateOverall,CumulResponseRateInternet,DailyResponseRateInternet,RESP_DATE,state,county,tract
1169,1400000US24019970900,"Census Tract 9709, Dorchester County, Maryland",47.5,0.3,15.1,0,2020-08-18,24,019,970900
83,1400000US24039930200,"Census Tract 9302, Somerset County, Maryland",48.1,0.2,17.8,0.1,2020-08-18,24,039,930200
87,1400000US24039930600,"Census Tract 9306, Somerset County, Maryland",47.4,0.2,18.1,0.1,2020-08-18,24,039,930600
330,1400000US24019970100,"Census Tract 9701, Dorchester County, Maryland",53.9,0.2,18.6,0,2020-08-18,24,019,970100
246,1400000US24015030100,"Census Tract 301, Cecil County, Maryland",42.9,0,18.8,0,2020-08-18,24,015,030100
377,1400000US24019970804,"Census Tract 9708.04, Dorchester County, Maryland",49,0.4,19.6,0.1,2020-08-18,24,019,970804
118,1400000US24045000300,"Census Tract 3, Wicomico County, Maryland",37.8,0.3,20.3,0.1,2020-08-18,24,045,000300
86,1400000US24039930500,"Census Tract 9305, Somerset County, Maryland",58.9,0,20.5,0,2020-08-18,24,039,930500
375,1400000US24019970500,"Census Tract 9705, Dorchester County, Maryland",54,0.4,21.5,0,2020-08-18,24,019,970500
173,1400000US24047950700,"Census Tract 9507, Worcester County, Maryland",47.1,0,21.6,0,2020-08-18,24,047,950700


In [13]:
# Combine state, county, tract to get FIPS code for join to tract spatial data.
cols = ['state', 'county', 'tract']
easternMD_df['FIPS'] = easternMD_df[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
easternMD_df

,GEO_ID,NAME,CumulResponseRateOverall,DailyResponseRateOverall,CumulResponseRateInternet,DailyResponseRateInternet,RESP_DATE,state,county,tract,FIPS
1169,1400000US24019970900,"Census Tract 9709, Dorchester County, Maryland",47.5,0.3,15.1,0,2020-08-18,24,019,970900,24019970900
83,1400000US24039930200,"Census Tract 9302, Somerset County, Maryland",48.1,0.2,17.8,0.1,2020-08-18,24,039,930200,24039930200
87,1400000US24039930600,"Census Tract 9306, Somerset County, Maryland",47.4,0.2,18.1,0.1,2020-08-18,24,039,930600,24039930600
330,1400000US24019970100,"Census Tract 9701, Dorchester County, Maryland",53.9,0.2,18.6,0,2020-08-18,24,019,970100,24019970100
246,1400000US24015030100,"Census Tract 301, Cecil County, Maryland",42.9,0,18.8,0,2020-08-18,24,015,030100,24015030100
...,...,...,...,...,...,...,...,...,...,...,...
296,1400000US24015030700,"Census Tract 307, Cecil County, Maryland",76.5,0.2,66.8,0.1,2020-08-18,24,015,030700,24015030700
94,1400000US24041960502,"Census Tract 9605.02, Talbot County, Maryland",82.4,0.1,70.3,0,2020-08-18,24,041,960502,24041960502
166,1400000US24045010605,"Census Tract 106.05, Wicomico County, Maryland",81.4,0.3,72.5,0,2020-08-18,24,045,010605,24045010605
169,1400000US24047950000,"Census Tract 9500, Worcester County, Maryland",11.5,0.1,9.7,0,2020-08-18,24,047,950000,24047950000


### Combine and explore relationships

In [14]:
# Merge response rates with ACS 5-yr demographics
easternMD_ACS_Census2020 = easternMD_ACS_df.merge(easternMD_df, left_on='tract', right_on='tract')
easternMD_ACS_Census2020

,TotEst_65Years,TotPerc_65Years,TotEst_LangNotEnglish,TotPercLangNotEnglish,TotEst_HHwithBroadband,TotPerc_HHwithBroadband,TotEst_25YearsWithHSEd,TotPerc_25YearsWithHSEd,TotEst_withDisability,TotPerc_withDisability,...,GEO_ID,NAME,CumulResponseRateOverall,DailyResponseRateOverall,CumulResponseRateInternet,DailyResponseRateInternet,RESP_DATE,state_y,county_y,FIPS
0,457,14.1,162,5.3,1021,80.6,869,39.9,557,18.4,...,1400000US24011955302,"Census Tract 9553.02, Caroline County, Maryland",67.2,0.2,54.4,0.1,2020-08-18,24,011,24011955302
1,583,19.6,105,3.7,1038,92.6,563,26.4,333,11.2,...,1400000US24035810902,"Census Tract 8109.02, Queen Anne's County, Mar...",73.1,0.1,63.6,0,2020-08-18,24,035,24035810902
2,691,16.0,387,9.7,1093,66.0,1256,45.5,837,19.4,...,1400000US24011955600,"Census Tract 9556, Caroline County, Maryland",60.4,0.3,45.9,0.2,2020-08-18,24,011,24011955600
3,792,20.5,177,4.8,1097,69.6,1257,42.7,625,16.7,...,1400000US24039930102,"Census Tract 9301.02, Somerset County, Maryland",58.8,0.2,45.7,0.1,2020-08-18,24,039,24039930102
4,835,13.3,1637,28.6,1128,54.4,1240,32.6,551,9.2,...,1400000US24045000100,"Census Tract 1, Wicomico County, Maryland",52,0.2,25.9,0.1,2020-08-18,24,045,24045000100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,801,33.1,172,7.5,970,90.0,300,15.9,372,15.4,...,1400000US24047950700,"Census Tract 9507, Worcester County, Maryland",47.1,0,21.6,0,2020-08-18,24,047,24047950700
99,906,28.8,86,2.9,978,78.2,767,33.9,470,15.0,...,1400000US24029950400,"Census Tract 9504, Kent County, Maryland",59.1,0.2,24,0,2020-08-18,24,029,24029950400
100,906,28.8,86,2.9,978,78.2,767,33.9,470,15.0,...,1400000US24047950400,"Census Tract 9504, Worcester County, Maryland",57.1,0.2,50.7,0.1,2020-08-18,24,047,24047950400
101,592,20.4,203,7.3,978,89.6,438,22.0,302,10.4,...,1400000US24029950400,"Census Tract 9504, Kent County, Maryland",59.1,0.2,24,0,2020-08-18,24,029,24029950400


In [15]:
# Add field for pie chart
easternMD_ACS_Census2020['CumulResponseRateOverall'] = easternMD_ACS_Census2020['CumulResponseRateOverall'].astype(str).astype(float)
easternMD_ACS_Census2020['CumulResponseRateInternet'] = easternMD_ACS_Census2020['CumulResponseRateInternet'].astype(str).astype(float)
easternMD_ACS_Census2020['RESP_DATE'] = pd.to_datetime(easternMD_ACS_Census2020['RESP_DATE'])
easternMD_ACS_Census2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 24 columns):
TotEst_65Years               103 non-null object
TotPerc_65Years              103 non-null object
TotEst_LangNotEnglish        103 non-null object
TotPercLangNotEnglish        103 non-null object
TotEst_HHwithBroadband       103 non-null object
TotPerc_HHwithBroadband      103 non-null object
TotEst_25YearsWithHSEd       103 non-null object
TotPerc_25YearsWithHSEd      103 non-null object
TotEst_withDisability        103 non-null object
TotPerc_withDisability       103 non-null object
TotPerc_Poverty              103 non-null object
state_x                      103 non-null object
county_x                     103 non-null object
tract                        103 non-null object
GEO_ID                       103 non-null object
NAME                         103 non-null object
CumulResponseRateOverall     103 non-null float64
DailyResponseRateOverall     103 non-null object
CumulRespons

In [16]:
easternMD_ACS_Census2020['CumulResponseRateMailPhone'] = easternMD_ACS_Census2020['CumulResponseRateOverall'] - easternMD_ACS_Census2020['CumulResponseRateInternet'] 

In [17]:
easternMD_ACS_Census2020['NoResponse'] = 100 - easternMD_ACS_Census2020['CumulResponseRateOverall']

In [18]:
easternMD_ACS_Census2020.columns

Index(['TotEst_65Years', 'TotPerc_65Years', 'TotEst_LangNotEnglish',
       'TotPercLangNotEnglish', 'TotEst_HHwithBroadband',
       'TotPerc_HHwithBroadband', 'TotEst_25YearsWithHSEd',
       'TotPerc_25YearsWithHSEd', 'TotEst_withDisability',
       'TotPerc_withDisability', 'TotPerc_Poverty', 'state_x', 'county_x',
       'tract', 'GEO_ID', 'NAME', 'CumulResponseRateOverall',
       'DailyResponseRateOverall', 'CumulResponseRateInternet',
       'DailyResponseRateInternet', 'RESP_DATE', 'state_y', 'county_y', 'FIPS',
       'CumulResponseRateMailPhone', 'NoResponse'],
      dtype='object', name=0)

In [19]:
easternMD_matrix = easternMD_ACS_Census2020.drop(['FIPS', 
                                                  'state_y',
                                                  'county_y',
                                                  'RESP_DATE',
                                                  'GEO_ID',
                                                  'NAME',
                                                  'tract',
                                                  'state_x',
                                                  'county_x',
                                                  'DailyResponseRateInternet',
                                                  'DailyResponseRateOverall'], axis=1)
easternMD_matrix

,TotEst_65Years,TotPerc_65Years,TotEst_LangNotEnglish,TotPercLangNotEnglish,TotEst_HHwithBroadband,TotPerc_HHwithBroadband,TotEst_25YearsWithHSEd,TotPerc_25YearsWithHSEd,TotEst_withDisability,TotPerc_withDisability,TotPerc_Poverty,CumulResponseRateOverall,CumulResponseRateInternet,CumulResponseRateMailPhone,NoResponse
0,457,14.1,162,5.3,1021,80.6,869,39.9,557,18.4,20.3,67.2,54.4,12.8,32.8
1,583,19.6,105,3.7,1038,92.6,563,26.4,333,11.2,2.3,73.1,63.6,9.5,26.9
2,691,16.0,387,9.7,1093,66.0,1256,45.5,837,19.4,21.5,60.4,45.9,14.5,39.6
3,792,20.5,177,4.8,1097,69.6,1257,42.7,625,16.7,15.2,58.8,45.7,13.1,41.2
4,835,13.3,1637,28.6,1128,54.4,1240,32.6,551,9.2,30.5,52.0,25.9,26.1,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,801,33.1,172,7.5,970,90.0,300,15.9,372,15.4,4.6,47.1,21.6,25.5,52.9
99,906,28.8,86,2.9,978,78.2,767,33.9,470,15.0,6.0,59.1,24.0,35.1,40.9
100,906,28.8,86,2.9,978,78.2,767,33.9,470,15.0,6.0,57.1,50.7,6.4,42.9
101,592,20.4,203,7.3,978,89.6,438,22.0,302,10.4,4.8,59.1,24.0,35.1,40.9


In [20]:
# Explore independent variable (response rates) relationships to dependent variables (ACS data).
# Principle component analysis? R2 matrix?

### Export to feature layer for web app builder development

In [21]:
# Authenticate GIS.
gis = GIS("pro")
print("Successfully logged in as: " + gis.properties.user.username)

Successfully logged in as: tlombar4_GISandData


In [22]:
# Query US Census Tracts to the the State of Maryland.
f1 = FeatureLayer("https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Census_Tract_Areas_analysis_trim/FeatureServer/0")
MDTracts_sdf = f1.query(where="STATE = 'MD'", out_sr=4326, as_df=True)
MDTracts_sdf

,OBJECTID,FIPS,COUNTY,STATE,POPULATION,SQMI,Shape__Area,Shape__Length,SHAPE
0,58445,24023000500,Garrett,MD,3973,74.96,3.266585e+08,85532.074421,"{'rings': [[[-79.3372059228647, 39.59281694827..."
1,58446,24023000600,Garrett,MD,6284,83.65,3.638154e+08,117593.765915,"{'rings': [[[-79.3923079301737, 39.57123293310..."
2,58447,24023000700,Garrett,MD,5710,82.81,3.588558e+08,101242.260314,"{'rings': [[[-79.3419113548345, 39.42422365906..."
3,58449,24001002100,Allegany,MD,3340,24.63,1.074261e+08,50426.634470,"{'rings': [[[-78.9669970831679, 39.59859406816..."
4,58450,24001002200,Allegany,MD,3713,23.84,1.037783e+08,58407.054870,"{'rings': [[[-79.0352160691641, 39.52142200880..."
...,...,...,...,...,...,...,...,...,...
1389,61681,24015030501,Cecil,MD,4299,11.38,4.980721e+07,38002.450323,"{'rings': [[[-75.8531869622967, 39.66378200919..."
1390,61682,24015030503,Cecil,MD,6320,16.38,7.157559e+07,70383.243900,"{'rings': [[[-75.8448850580107, 39.58100899325..."
1391,61683,24015030506,Cecil,MD,8553,4.18,1.827290e+07,21701.578634,"{'rings': [[[-75.7961959629756, 39.64008102598..."
1392,61684,24015030601,Cecil,MD,3837,20.34,8.907897e+07,47504.563426,"{'rings': [[[-75.8536629709016, 39.72195300175..."


In [23]:
# Merge eastern shore MD tracts Spatially Enabled DataFrame with response rate and dependent variables.
easternMD_ACS_Census2020_sdf = MDTracts_sdf.merge(easternMD_ACS_Census2020, left_on='FIPS', right_on='FIPS')
easternMD_ACS_Census2020_sdf

,OBJECTID,FIPS,COUNTY,STATE,POPULATION,SQMI,Shape__Area,Shape__Length,SHAPE,TotEst_65Years,...,NAME,CumulResponseRateOverall,DailyResponseRateOverall,CumulResponseRateInternet,DailyResponseRateInternet,RESP_DATE,state_y,county_y,CumulResponseRateMailPhone,NoResponse
0,60336,24019970900,Dorchester,MD,1514,231.90,9.791462e+08,539805.404343,"{'rings': [[[-76.0918389238541, 38.11849194313...",387,...,"Census Tract 9709, Dorchester County, Maryland",47.5,0.3,15.1,0,2020-08-18,24,019,32.4,52.5
1,60337,24039930500,Somerset,MD,4462,81.30,3.397598e+08,286287.661974,"{'rings': [[[-75.8784149169353, 37.91372199995...",1037,...,"Census Tract 9305, Somerset County, Maryland",58.9,0,20.5,0,2020-08-18,24,039,38.4,41.1
2,60338,24039930600,Somerset,MD,2917,8.71,3.639426e+07,81057.312150,"{'rings': [[[-75.8779659846627, 37.96926307840...",621,...,"Census Tract 9306, Somerset County, Maryland",47.4,0.2,18.1,0.1,2020-08-18,24,039,29.3,52.6
3,60339,24039930200,Somerset,MD,1771,51.72,2.168345e+08,479330.836903,"{'rings': [[[-76.0200659185562, 37.95574095756...",507,...,"Census Tract 9302, Somerset County, Maryland",48.1,0.2,17.8,0.1,2020-08-18,24,039,30.3,51.9
4,60340,24047951500,Worcester,MD,4819,27.55,1.152219e+08,55939.747428,"{'rings': [[[-75.5423209186479, 38.04237803352...",888,...,"Census Tract 9515, Worcester County, Maryland",61.7,0.1,48.2,0,2020-08-18,24,047,13.5,38.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,61680,24015030400,Cecil,MD,4747,1.86,8.139551e+06,16452.592478,"{'rings': [[[-75.8271859950891, 39.61780004596...",677,...,"Census Tract 304, Cecil County, Maryland",51.2,0.3,22.9,0,2020-08-18,24,015,28.3,48.8
99,61681,24015030501,Cecil,MD,4299,11.38,4.980721e+07,38002.450323,"{'rings': [[[-75.8531869622967, 39.66378200919...",645,...,"Census Tract 305.01, Cecil County, Maryland",70.3,0.4,61.5,0.1,2020-08-18,24,015,8.8,29.7
100,61682,24015030503,Cecil,MD,6320,16.38,7.157559e+07,70383.243900,"{'rings': [[[-75.8448850580107, 39.58100899325...",567,...,"Census Tract 305.03, Cecil County, Maryland",59.2,0.4,51.6,0.2,2020-08-18,24,015,7.6,40.8
101,61684,24015030601,Cecil,MD,3837,20.34,8.907897e+07,47504.563426,"{'rings': [[[-75.8536629709016, 39.72195300175...",533,...,"Census Tract 306.01, Cecil County, Maryland",75.0,0,65.4,0,2020-08-18,24,015,9.6,25.0


In [24]:
# Add content
lyr = easternMD_ACS_Census2020_sdf.spatial.to_featurelayer('easternMD_Census2020Response_enriched')
lyr

<Item title:"easternMD_Census2020Response_enriched" type:Feature Layer Collection owner:tlombar4_GISandData>

In [25]:
easternMD_ACS_Census2020_sdf.columns

Index(['OBJECTID', 'FIPS', 'COUNTY', 'STATE', 'POPULATION', 'SQMI',
       'Shape__Area', 'Shape__Length', 'SHAPE', 'TotEst_65Years',
       'TotPerc_65Years', 'TotEst_LangNotEnglish', 'TotPercLangNotEnglish',
       'TotEst_HHwithBroadband', 'TotPerc_HHwithBroadband',
       'TotEst_25YearsWithHSEd', 'TotPerc_25YearsWithHSEd',
       'TotEst_withDisability', 'TotPerc_withDisability', 'TotPerc_Poverty',
       'state_x', 'county_x', 'tract', 'GEO_ID', 'NAME',
       'CumulResponseRateOverall', 'DailyResponseRateOverall',
       'CumulResponseRateInternet', 'DailyResponseRateInternet', 'RESP_DATE',
       'state_y', 'county_y', 'CumulResponseRateMailPhone', 'NoResponse'],
      dtype='object')